In [1]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
# import tensorflow as tf
import pandas as pd

In [8]:
import os
import pandas as pd

# Pfad zum Verzeichnis, das die Attitude Roots enthält
root_dir = r'\data'  

# Liste für die Ergebnisse
data = []

# Durchlaufe alle Unterverzeichnisse im Root-Verzeichnis
for attitude_root in os.listdir(root_dir):
    attitude_path = os.path.join(root_dir, attitude_root)
    # Prüfe, ob es sich um ein Verzeichnis handelt
    if os.path.isdir(attitude_path):
        review_path = os.path.join(attitude_path, 'review')
        # Prüfe, ob der 'review'-Ordner existiert
        if os.path.exists(review_path):
            # Durchlaufe alle Dateien im 'review'-Ordner
            for filename in os.listdir(review_path):
                file_path = os.path.join(review_path, filename)
                # Nur Dateien berücksichtigen
                if os.path.isfile(file_path):
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                        # Sätze durch Zeilenumbrüche getrennt
                        sentences = content.strip().split('\n')
                        for sentence in sentences:
                            # Überspringe leere Zeilen
                            if sentence.strip():
                                data.append({
                                    'Satz': sentence.strip(),
                                    'Label': attitude_root
                                })

df = pd.DataFrame(data)

print(df)


FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: '\\data'

In [ ]:
df = x
df[‘labels’].unique()




: 

In [ ]:
#Encode label for easy identification.
 df[‘encoded_cat’] = df[‘labels’].astype(‘category’).cat.codes
 print(df.head())

 data_texts = df[“data”].to_list() # Features (not tokenized yet)
 data_labels = df[“encoded_cat”].to_list() # Labels

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
dict(train_encodings),
train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
dict(val_encodings),
val_labels
))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
 
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

In [ ]:
#from tensorflow.keras.callbacks import EarlyStopping
 
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
 
model.fit(train_dataset.shuffle(1000).batch(16),
epochs=2,
batch_size=16,
validation_data=val_dataset.shuffle(1000).batch(16),
callbacks=[early_stopping])

In [ ]:
from tensorflow.keras.models import load_model
save_directory = "Multitext_Classification_colab" # Change this to your preferred location
 
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
# Evaluate

save_directory = "Multitext_Classification_2"
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

